In [1]:
import fsc
from fsc.devices import w7x
import numpy as np

In [2]:
fsc.resolve.importOfflineData("C:\\Users\\Knieps\\Downloads\\w7x.fsc")

In [3]:
tracer = fsc.tracer()

In [ ]:
geoGrid = w7x.defaultGeometryGrid
geometry = tracer.indexGeometry(w7x.divertor(), grid = geoGrid)
geometry

In [4]:
geoGrid = w7x.defaultGeometryGrid
geometry = fsc.Geometry(fsc.data.readArchive('geometry.fsc'))

In [5]:
fsc.data.writeArchive(geometry.geometry, 'geometry.fsc')

In [6]:
help(geometry)

Help on Geometry in module fsc object:

class Geometry(builtins.object)
 |  Geometry(geo=None)
 |  
 |  Methods defined here:
 |  
 |  __add__(self, other)
 |  
 |  __init__(self, geo=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  graph(self, **kwargs)
 |  
 |  ptree(self)
 |  
 |  resolve(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  geometry



In [5]:
archiveCoils = fsc.data.readArchive("cadCoils.fsc")
config = w7x.standard(coils = archiveCoils)
grid = w7x.defaultGrid.clone()
grid.nR = 128
grid.nZ = 128
grid.nPhi = 32

In [6]:
import matplotlib.pyplot as plt
import numpy as np

In [7]:
tracer.trace([6.5, 0,0], config, grid = grid, geometryGrid = geoGrid, geometry = geometry, distanceLimit = 1e2)

Computed field obtained
Indexed geometry obtained
Starting trace


namespace(endPoints=array([-5.11769073,  2.50216355,  1.50072312]))

In [ ]:
points = np.linspace([5.6, 0.0, 0.0], [6.0, 0, 0], 10, axis = 1)
pcPoints = tracer.poincareInPhiPlanes(points, [0.0], 2000, config, grid, distanceLimit = 5e4, stepSize = 1e-3)

Assigning
Tracing
